In [ ]:
import os
import re
import boto3
import pandas as pd
from io import BytesIO

BUCKET = os.getenv("AWS_S3_BUCKET_NAME", "arcana-fiap")
REGION = os.getenv("AWS_REGION", "us-east-2")

def get_s3():
    return boto3.client(
        "s3",
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region_name=REGION,
    )

def listar_parquets(bucket=BUCKET, prefix="silver/"):
    s3 = get_s3()
    keys = []
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            if obj["Key"].endswith(".parquet"):
                keys.append(obj["Key"])
    return keys

def nome_variavel(basename, existentes):
    """
    Converte 'meu-arquivo 1.parquet' -> 'meu_arquivo_1'
    Garante que não começa com número e não colide com nomes já usados.
    """
    name = os.path.splitext(basename)[0]
    name = re.sub(r"\W", "_", name)
    if re.match(r"^\d", name):
        name = f"df_{name}"
    original = name
    i = 1
    while name in existentes:
        name = f"{original}_{i}"
        i += 1
    return name

def carregar_parquets_em_variaveis(bucket=BUCKET, prefix="silver/", nomes=None, destino=None):
    """
    Carrega parquets do S3 e cria variáveis DataFrame com o nome do arquivo (normalizado).
    - nomes: lista de nomes (sem .parquet). Se None, carrega todos do prefixo.
    - destino: dict onde criar as variáveis (por padrão, globals()).
    Retorna um dict {nome_variavel: 's3://bucket/key'} com o mapeamento criado.
    """
    s3 = get_s3()
    destino = destino if destino is not None else globals()

    if nomes is None:
        keys = listar_parquets(bucket, prefix)
    else:
        keys = [f"{prefix}{n}.parquet" for n in nomes]

    if not keys:
        print("Nenhum parquet encontrado.")
        return {}

    criados = {}
    existentes = set(destino.keys())
    for key in keys:
        try:
            obj = s3.get_object(Bucket=bucket, Key=key)
            df = pd.read_parquet(BytesIO(obj["Body"].read()))
            var_name = nome_variavel(os.path.basename(key), existentes)
            destino[var_name] = df
            existentes.add(var_name)
            criados[var_name] = f"s3://{bucket}/{key}"
            print(f"{var_name} ← {key}  | {len(df)} linhas")
        except s3.exceptions.NoSuchKey:
            print(f"Não encontrado: s3://{bucket}/{key}")
        except Exception as e:
            print(f"Erro lendo {key}: {e}")

    return criados

In [11]:
carregar_parquets_em_variaveis(prefix="silver/")

✅ clientes_desde_1 ← silver/clientes_desde.parquet  | 10615 linhas
✅ contratacoes_ultimos_12_meses_1 ← silver/contratacoes_ultimos_12_meses.parquet  | 4314 linhas
✅ dados_clientes_1 ← silver/dados_clientes.parquet  | 238597 linhas
✅ historico_1 ← silver/historico.parquet  | 22740 linhas
✅ mrr_1 ← silver/mrr.parquet  | 7309 linhas
✅ nps_relacional_1 ← silver/nps_relacional.parquet  | 14143 linhas
✅ nps_transacional_aquisicao_1 ← silver/nps_transacional_aquisicao.parquet  | 178 linhas
✅ nps_transacional_implantacao_1 ← silver/nps_transacional_implantacao.parquet  | 662 linhas
✅ nps_transacional_onboarding_1 ← silver/nps_transacional_onboarding.parquet  | 208 linhas
✅ nps_transacional_produto_1 ← silver/nps_transacional_produto.parquet  | 113207 linhas
✅ nps_transacional_suporte_1 ← silver/nps_transacional_suporte.parquet  | 74794 linhas
✅ telemetria_1_1 ← silver/telemetria_1.parquet  | 2535221 linhas
✅ telemetria_10_1 ← silver/telemetria_10.parquet  | 1884789 linhas
✅ telemetria_11_1 ← s

{'clientes_desde_1': 's3://arcana-fiap/silver/clientes_desde.parquet',
 'contratacoes_ultimos_12_meses_1': 's3://arcana-fiap/silver/contratacoes_ultimos_12_meses.parquet',
 'dados_clientes_1': 's3://arcana-fiap/silver/dados_clientes.parquet',
 'historico_1': 's3://arcana-fiap/silver/historico.parquet',
 'mrr_1': 's3://arcana-fiap/silver/mrr.parquet',
 'nps_relacional_1': 's3://arcana-fiap/silver/nps_relacional.parquet',
 'nps_transacional_aquisicao_1': 's3://arcana-fiap/silver/nps_transacional_aquisicao.parquet',
 'nps_transacional_implantacao_1': 's3://arcana-fiap/silver/nps_transacional_implantacao.parquet',
 'nps_transacional_onboarding_1': 's3://arcana-fiap/silver/nps_transacional_onboarding.parquet',
 'nps_transacional_produto_1': 's3://arcana-fiap/silver/nps_transacional_produto.parquet',
 'nps_transacional_suporte_1': 's3://arcana-fiap/silver/nps_transacional_suporte.parquet',
 'telemetria_1_1': 's3://arcana-fiap/silver/telemetria_1.parquet',
 'telemetria_10_1': 's3://arcana-fia

In [15]:
df_telemetria

,COD_CLIENTE,DURACAO_EVENTO,ID_MODULO,ID_LINHA_PRODUTO,DATA_REFERENCIA,ID_SLOT,STATUS_LICENCA,CLOUD,CLIENTE_PRIME
0,TEXKCV00,580.724,6,2,2025-03-19,4133,Desconectado,NaN,NaN
1,TAAHTU00,414787.069,6,2,2025-03-19,4000,Desconectado,NaN,NaN
2,TEXKCV00,4933.511,6,2,2025-03-19,4133,Desconectado,NaN,NaN
3,TEWERU00,414086.474,7,2,2025-03-19,4000,Desconectado,NaN,NaN
4,TFDFHJ00,0.000,524,3,2025-03-19,0,Desconectado,NaN,NaN
...,...,...,...,...,...,...,...,...,...
30804192,TDC0GU00,180.000,726,9,2025-03-18,3502,Desconectado,NaN,NaN
30804193,TDC0GU00,120.000,726,9,2025-03-18,3502,Desconectado,NaN,NaN
30804194,T4645700,1349.000,534,3,2025-03-18,534,Desconectado,NaN,NaN
30804195,T4308700,61.000,534,3,2025-03-18,534,Desconectado,NaN,NaN
